In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import torch

In [3]:
import sys

prosst_path = "/home/rcalef/sandbox/repos/external/ProSST"
sys.path.append(prosst_path)

In [4]:
from prosst.structure.get_sst_seq import SSTPredictor

In [5]:
example_pdb = "/weka/scratch/weka/kellislab/rcalef/data/pdb_alphafolddb/AF-A1RZJ9-F1-model_v4.pdb"
predictor = SSTPredictor(structure_vocab_size=2048) # can be 20, 128, 512, 1024, 2048, 4096
result = predictor.predict_from_pdb(example_pdb)

---------- Load Model on cuda ----------
MODEL: 5.90M parameters
---------- Building Subgraphs ----------


100%|██████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.88s/it]


In [6]:
result[0]

{'name': 'AF-A1RZJ9-F1-model_v4.pdb',
 'aa_seq': 'MRAELPKRVVVERGALQFLPEVLRELGCSKTVVVTDSGVWSVVGSVVEGALRGLAYEVVYIEAADNSNVERARSAARRVEACAVAGLGGGRPVDVAKYAAFMEGLPFVSVPTAISHDGFASPIVALKDPEGNPLSIFTRPPAAVLVDLAVVSRAPRRLLASGVGDIVGKVTSVADARLAQRLTGEEVPEVALRMAETAARMVLDEVDEIASWTERGVGVLAQAGLLAGMAMAVAGSSRPCSGSEHLFSHSLDKYVPWKKSLHGEQVGVGAIIASYLHGFNWRVIRDALAKVGAPTTVEGLGVTGEDAVRALLKARELRKRFTILDVVELNEGLAWKVLRETGVAPTA',
 '2048_sst_seq': [1534,
  1560,
  146,
  1590,
  466,
  973,
  1423,
  246,
  314,
  1155,
  565,
  1683,
  352,
  1348,
  336,
  196,
  699,
  913,
  2037,
  1282,
  169,
  235,
  259,
  937,
  1304,
  235,
  388,
  1612,
  1232,
  157,
  1751,
  1417,
  1393,
  269,
  8,
  1336,
  646,
  397,
  83,
  397,
  64,
  967,
  1960,
  624,
  881,
  513,
  174,
  1207,
  75,
  174,
  1380,
  1176,
  510,
  727,
  1359,
  774,
  2002,
  774,
  1089,
  1185,
  1838,
  505,
  372,
  1838,
  372,
  1687,
  750,
  573,
  1339,
  1739,
  1339,
  2037,
  1663,
  1691,
  561,
  365,
  1938,
  2000,
  938,
 

In [7]:
len(result[0]["2048_sst_seq"])

347

In [9]:
len(result[0]["aa_seq"])

347

In [10]:
from transformers import AutoModelForMaskedLM, AutoTokenizer


In [11]:
model_path = "/home/rcalef/storage/om_storage/model_weights/ProSST-2048"
model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [12]:
model

ProSSTForMaskedLM(
  (prosst): ProSSTModel(
    (embeddings): ProSSTEmbeddings(
      (word_embeddings): Embedding(25, 768, padding_idx=0)
      (ss_embeddings): Embedding(2051, 768)
      (ss_layer_norm): ProSSTLayerNorm()
      (LayerNorm): ProSSTLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ProSSTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ProSSTLayer(
          (attention): ProSSTAttention(
            (self): DisentangledSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (ss_proj): Linear(in_features=768, out_features=

In [13]:
tokenizer

EsmTokenizer(name_or_path='/home/rcalef/storage/om_storage/model_weights/ProSST-2048', vocab_size=25, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	23: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	24: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [32]:
aa_toks = tokenizer(result[0]["aa_seq"], return_tensors="pt")
aa_toks

{'input_ids': tensor([[ 1, 13, 17,  3,  6, 12, 15, 11, 17, 20, 20, 20,  6, 17,  8,  3, 12, 16,
          7, 12, 15,  6, 20, 12, 17,  6, 12,  8,  4, 18, 11, 19, 20, 20, 20, 19,
          5, 18,  8, 20, 21, 18, 20, 20,  8, 18, 20, 20,  6,  8,  3, 12, 17,  8,
         12,  3, 22,  6, 20, 20, 22, 10,  6,  3,  3,  5, 14, 18, 14, 20,  6, 17,
          3, 17, 18,  3,  3, 17, 17, 20,  6,  3,  4,  3, 20,  3,  8, 12,  8,  8,
          8, 17, 15, 20,  5, 20,  3, 11, 22,  3,  3,  7, 13,  6,  8, 12, 15,  7,
         20, 18, 20, 15, 19,  3, 10, 18,  9,  5,  8,  7,  3, 18, 15, 10, 20,  3,
         12, 11,  5, 15,  6,  8, 14, 15, 12, 18, 10,  7, 19, 17, 15, 15,  3,  3,
         20, 12, 20,  5, 12,  3, 20, 20, 18, 17,  3, 15, 17, 17, 12, 12,  3, 18,
          8, 20,  8,  5, 10, 20,  8, 11, 20, 19, 18, 20,  3,  5,  3, 17, 12,  3,
         16, 17, 12, 19,  8,  6,  6, 20, 15,  6, 20,  3, 12, 17, 13,  3,  6, 19,
          3,  3, 17, 13, 20, 12,  5,  6, 20,  5,  6, 10,  3, 18, 21, 19,  6, 17,
          8, 2

In [33]:
aa_toks["input_ids"].shape

torch.Size([1, 349])

In [37]:
ss_toks = torch.tensor(result[0]["2048_sst_seq"])
ss_toks.shape

torch.Size([347])

In [39]:
torch.cat([1], ss_toks, [2])

TypeError: cat() received an invalid combination of arguments - got (list, Tensor, list), but expected one of:
 * (tuple of Tensors tensors, int dim = 0, *, Tensor out = None)
 * (tuple of Tensors tensors, name dim, *, Tensor out = None)


In [29]:
aa_toks["input_ids"].shape

torch.Size([1, 347])

In [31]:
# def forward(
#     self,
#     input_ids: Optional[torch.Tensor] = None,
#     ss_input_ids: Optional[torch.Tensor] = None,
#     attention_mask: Optional[torch.Tensor] = None,
#     token_type_ids: Optional[torch.Tensor] = None,
#     position_ids: Optional[torch.Tensor] = None,
#     inputs_embeds: Optional[torch.Tensor] = None,
#     labels: Optional[torch.Tensor] = None,
#     output_attentions: Optional[bool] = None,
#     output_hidden_states: Optional[bool] = None,
#     return_dict: Optional[bool] = None,
# ) -> Union[Tuple, MaskedLMOutput]:
with torch.inference_mode():
    out = model(
        input_ids=aa_toks["input_ids"],
        ss_input_ids=ss_toks,
        attention_mask=aa_toks["attention_mask"],
    )
out

MaskedLMOutput(loss=None, logits=tensor([[[-18.3368, -12.2662, -13.9507,  ...,   0.8657, -15.6547, -18.3795],
         [-18.8052, -12.7068, -15.0800,  ...,   0.6976, -16.0266, -18.8201],
         [-18.6050, -12.5979, -14.4972,  ...,   0.2606, -15.8135, -18.6339],
         ...,
         [-18.6463, -12.9135, -14.2035,  ...,   0.7077, -15.9107, -18.6853],
         [-18.7029, -13.0057, -14.9421,  ...,   0.2711, -16.3094, -18.7389],
         [-19.0306, -13.0453, -15.0684,  ...,   0.3415, -16.1042, -19.0603]]]), hidden_states=None, attentions=None)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("AI4Protein/ProSST-2048", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("AI4Protein/ProSST-2048", trust_remote_code=True)

# Test modules

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os

import torch

In [3]:
from magneton.config import (
    DataConfig,
    EmbeddingConfig,
    ModelConfig,
    TrainingConfig,
    PipelineConfig,
)
from magneton.embedders.prosst_embedder import ProSSTConfig, ProSSTDataModule, ProSSTEmbedder, move_prosst_inputs_to_device
from magneton.embedders.esmc_embedder import ESMCConfig, ESMCDataModule, ESMCEmbedder
from magneton.training.embedding_mlp import EmbeddingMLP, MultitaskEmbeddingMLP
from magneton.types import InterProType
from magneton.utils import move_inputs_to_device

/net/vast-storage/scratch/vast/kellislab/rcalef/sandbox/repos/magneton/magneton/external/ProSST


## Load data

In [4]:
interpro_path = "/weka/scratch/weka/kellislab/rcalef/data/interpro/103.0/"
fasta_path = "/weka/scratch/weka/kellislab/rcalef/data/uniprot/uniprot_sprot.fasta.gz"
labels_path = "/weka/scratch/weka/kellislab/rcalef/data/interpro/103.0/label_sets/selected_subset"
pickle_path = os.path.join(interpro_path, "swissprot", "sharded_swissprot", "with_ss", "debug_datasets")
#pickle_path = os.path.join(interpro_path, "swissprot", "sharded_swissprot", "with_ss", "dataset_splits", "seq_splits")

In [5]:
model_path = "/home/rcalef/storage/om_storage/model_weights/ProSST-2048"

data_config = DataConfig(
    data_dir=pickle_path,
    prefix="swissprot.with_ss",
    fasta_path=fasta_path,
    labels_path=labels_path,
    substruct_types=["Domain"],
    collapse_labels=True,
    struct_template="/weka/scratch/weka/kellislab/rcalef/data/pdb_alphafolddb/AF-%s-F1-model_v4.pdb",
    model_specific_params={
        "model_path": model_path,
        "num_threads": 4,
    }
)

train_config = TrainingConfig(
    batch_size=2,
)

In [6]:
data_module = ProSSTDataModule(
    data_config=data_config,
    train_config=train_config,
)

In [7]:
load = True
if load:
    data_loader = data_module.train_dataloader()
    print(len(data_loader.dataset))
    print(data_loader.dataset.substruct_parser.num_labels())
    num_labels = data_loader.dataset.substruct_parser.num_labels()
else:
    num_labels = 917

100%|██████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.95s/it]

3715
917


In [8]:
data_loader.dataset[10].sequence

tensor([ 1, 13, 13, 11,  7, 11, 15, 14, 16, 19, 17, 19, 22,  5, 17,  6,  8,  7,
        11, 11, 17,  3,  3,  4, 12,  4,  7, 17, 18,  6, 16,  6,  5,  6, 20, 12,
        12, 20, 18, 18, 18, 17, 22, 15,  5, 16, 21, 10, 20, 15,  8,  8,  8, 13,
         6, 15,  6,  6,  6, 15,  8,  8,  3,  3, 20, 17,  6, 20, 22,  6,  6,  3,
         8, 20, 11,  8, 11, 12,  8, 17, 12, 12,  8, 10,  7,  6, 16, 14, 16,  5,
        17, 11,  9, 17, 19, 22, 20, 22, 20, 12, 19, 20, 19,  6, 10, 12,  6,  5,
        21,  6,  5, 18, 20, 14, 10,  8, 17, 11, 17,  6, 21,  7, 11, 20,  6,  5,
         3, 10, 11, 20, 12, 16,  4,  9, 11, 15, 20,  9,  3,  6, 22, 12,  6, 11,
        12, 11, 12,  8,  4, 18, 15,  3, 14,  8, 14, 18, 19, 20, 15, 18, 12, 15,
         5, 14, 14,  3, 12,  7, 20, 19,  3,  3, 16, 19, 18,  8, 12, 15, 18, 18,
        20, 17,  2])

In [9]:
batch = next(iter(data_loader))
batch

ProSSTBatch(substructures=[[LabeledSubstructure(ranges=[tensor([ 41, 229])], label=767, element_type='Domain')], [LabeledSubstructure(ranges=[tensor([ 10, 103])], label=711, element_type='Domain')]], prot_ids=['A1CCU0', 'A1AE45'], batch_graphs=DataBatch(edge_index=[2, 115420], node_s=[11237, 20], node_v=[11237, 3, 3], edge_s=[115420, 32], edge_v=[115420, 1, 3], batch=[11237], ptr=[688]), tokenized_seqs=tensor([[ 1, 13, 20, 18,  7, 11, 22, 12,  7, 12,  3,  3, 18,  3, 12,  8,  3, 12,
          3,  3, 15, 20,  6, 20,  6,  6, 18, 18, 21,  7, 14,  6, 19,  3, 12,  9,
          6,  7,  3,  6, 17,  3,  8, 19, 15, 18, 18, 19,  8, 21, 14, 14,  8, 22,
         22, 22, 18,  7, 21, 19,  5, 14,  8,  8, 19, 20, 14, 22, 16, 14,  8, 14,
          8,  8, 18, 22, 18, 20, 16, 21, 11,  5, 19,  8, 14,  7, 20,  8,  8, 11,
          8, 21, 14, 15,  8, 18,  3, 17, 19, 10, 14, 22, 18,  8, 18,  7, 14, 15,
         18,  8, 14,  3, 22, 12, 19, 20, 22,  8, 21, 19, 19, 14, 15, 12, 20,  6,
         22, 22, 10, 20,  6

## Make model

In [10]:
model_config = ModelConfig(
    model_type="embedding_mlp",
    model_params={
        "hidden_dims": [256, 256],
        "dropout_rate": 0.1,
    },
    frozen_embedder=False,
)
prosst_config = ProSSTConfig(
    model_path=model_path,
)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
embedder = ProSSTEmbedder(prosst_config)

In [13]:
embedder = embedder.to(device)

In [14]:
batch = batch.to(device)

In [16]:
embedder.embed_batch(batch)

torch.Size([2, 460])
torch.Size([2, 458])
torch.Size([2, 460])


RuntimeError: The size of tensor a (458) must match the size of tensor b (460) at non-singleton dimension 1

In [26]:
embedder.encoder_model.W_v[0].scalar_norm.weight

Parameter containing:
tensor([0.3121, 0.3089, 0.3078, 0.3285, 0.2895, 0.2951, 0.3388, 0.2945, 0.2910,
        0.3344, 0.3081, 0.3292, 0.3346, 0.2586, 0.2828, 0.2632, 0.2735, 0.2689,
        0.3046, 0.3257], device='cuda:0', requires_grad=True)

In [ ]:
pipeline_config = PipelineConfig(
    data=data_config,
    model=model_config,
    training=train_config,
    embedding=EmbeddingConfig(
        model="esmc",
        model_params=esmc_config.__dict__,
    ),
)

# Train model
if data_config.collapse_labels:
    mlp = EmbeddingMLP(
        config=pipeline_config,
        num_classes=num_labels,
    )
else:
    mlp = MultitaskEmbeddingMLP(
        config=pipeline_config,
        num_classes=num_labels,
    )


=== Creating esmc embedder ===
EmbeddingConfig(_target_='magneton.config.EmbeddingConfig',
                model='esmc',
                batch_size=32,
                model_params={'mask_prob': 0.15,
                              'max_seq_length': 2048,
                              'model_size': '600m',
                              'rep_layer': 33,
                              'use_flash_attn': True,
                              'weights_path': '/weka/scratch/weka/kellislab/rcalef/model_weights/esmc-600m-2024-12'})
Config parameters: None
